In [1]:
%%capture
!pip install elasticsearch==8.8.0

In [2]:
%%bash

rm -rf elasticsearch*
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-8.8.0/
umount /sys/fs/cgroup
apt install cgroup-tools

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libcgroup1
The following NEW packages will be installed:
  cgroup-tools libcgroup1
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 109 kB of archives.
After this operation, 472 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcgroup1 amd64 0.41-10 [42.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 cgroup-tools amd64 0.41-10 [66.2 kB]
Fetched 109 kB in 0s (624 kB/s)
Selecting previously unselected package libcgroup1:amd64.
(Reading database ... 122541 files and directories currently installed.)
Preparing to unpack .../libcgroup1_0.41-10_amd64.deb ...
Unpacking libcgroup1:amd64 (0.41-10) ...
Selecting previously unselected package cgroup-tools.
Preparing to unpack .../cgroup-tools_0.41-10_amd64.deb ...
Unpacking cgroup-tools (0.41-10) ...
Setting up li

In [4]:
try:
    import os
    import elasticsearch
    from elasticsearch import Elasticsearch
    import numpy as np
    import pandas as pd
    import sys
    import json
    from ast import literal_eval
    from tqdm import tqdm 
    import datetime
    from elasticsearch import helpers
  
except Exception as e:
    print(f"error: {e}")    

In [5]:
%%bash --bg

sudo -H -u daemon elasticsearch-8.8.0/bin/elasticsearch

In [6]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(20)

In [7]:
!ps -ef | grep elastic

root        1734    1732  0 06:48 ?        00:00:00 sudo -H -u daemon elasticsearch-8.8.0/bin/elasticsearch
daemon      1735    1734 45 06:48 ?        00:00:11 /content/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/content/elasticsearch-8.8.0 -Des.path.conf=/content/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /content/elasticsearch-8.8.0/lib/*:/content/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon      1823    1735 99 06:48 ?        00:00:28 /content/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacit

In [8]:
!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

******************************************************************************
Note: The 'elasticsearch-setup-passwords' tool has been deprecated. This       command will be removed in a future release.
******************************************************************************

Initiating the setup of passwords for reserved users elastic,apm_system,kibana,kibana_system,logstash_system,beats_system,remote_monitoring_user.
The passwords will be randomly generated and printed to the console.
Please confirm that you would like to continue [y/N]y


Changed password for user apm_system
PASSWORD apm_system = 7H1LiHas7KFFJAgaAzP0

Changed password for user kibana_system
PASSWORD kibana_system = AHIKg9urTmCdY1zwQ29f

Changed password for user kibana
PASSWORD kibana = AHIKg9urTmCdY1zwQ29f

Changed password for user logstash_system
PASSWORD logstash_system = zf0oeamb7lfdjIejFQO3

Changed password for user beats_system
PASSWORD beats_system = KpUb1IAKV9lqDKUtDhVQ

Changed password for user rem

In [9]:
!curl --cacert /content/elasticsearch-8.8.0/config/certs/http_ca.crt -u elastic -H 'Content-Type: application/json' -XGET https://localhost:9200/?pretty=true

Enter host password for user 'elastic':
{
  "name" : "8b7bfbccbe50",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "g3UtfjQ9RFez1UqEXZxp0w",
  "version" : {
    "number" : "8.8.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "c01029875a091076ed42cdb3a41c10b1a9a5a20f",
    "build_date" : "2023-05-23T17:16:07.179039820Z",
    "build_snapshot" : false,
    "lucene_version" : "9.6.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [10]:
username = 'elastic'

password = 'DsWom9ixGWUM2R9kObKb'

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/content/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': '8b7bfbccbe50',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'g3UtfjQ9RFez1UqEXZxp0w',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}